In [ ]:
!wget https://www.dropbox.com/scl/fi/dwwd3r0gvxaiuq6pjnb9v/NF-ToN-IoT-V2.parquet?rlkey=kwfzpx81rpmahd61r9zkx2ct3&st=lfyqbgbt&dl=0 -O NF-ToN-IoT-V2.parquet

/bin/bash: line 1: -O: command not found
--2025-11-15 07:39:04--  https://www.dropbox.com/scl/fi/dwwd3r0gvxaiuq6pjnb9v/NF-ToN-IoT-V2.parquet?rlkey=kwfzpx81rpmahd61r9zkx2ct3
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucccad594f4cb7860055239560be.dl.dropboxusercontent.com/cd/0/inline/C1MsHCU9BgnvmMNqRewUIXYp3YuwpjyXAJT4AFsufXrGGuLE_Xvv57sFd7AAE2J3caHxDiy1lD9XLqKipXpKYhElv23LbTthXDe26VFKQpodz9P6v29TXzpaGNZFh8CWTlnpTeoU0qc12L2gdQL-Kc_y/file# [following]
--2025-11-15 07:39:05--  https://ucccad594f4cb7860055239560be.dl.dropboxusercontent.com/cd/0/inline/C1MsHCU9BgnvmMNqRewUIXYp3YuwpjyXAJT4AFsufXrGGuLE_Xvv57sFd7AAE2J3caHxDiy1lD9XLqKipXpKYhElv23LbTthXDe26VFKQpodz9P6v29TXzpaGNZFh8CWTlnpTeoU0qc12L2gdQL-Kc_y/file
Resolving ucccad594f4cb7860055239560be.dl.dropboxusercontent.com (ucccad594f4cb7860055239560

In [ ]:
!pip install --quiet pennylane torch scikit-learn pandas numpy pyarrow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 76.2 MB/s eta 0:00:00


In [ ]:
# Cell 1: imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import pennylane as qml
from pennylane import numpy as pnp
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from copy import deepcopy



/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


In [ ]:
# Cell 2: load parquet file (update path if needed)
# Put your NF-ToN-IoT-v2.parquet in Colab /content or mount Drive and use path.
parquet_path = "/content/NF-ToN-IoT-V2.parquet?rlkey=kwfzpx81rpmahd61r9zkx2ct3"  # <- update if needed
df = pd.read_parquet(parquet_path)

# Quick peek
print("Rows:", len(df))
print("Columns:", df.columns.tolist())
df.head().T  # show transpose for readability


Rows: 13135881
Columns: ['L4_SRC_PORT', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS', 'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'DURATION_IN', 'DURATION_OUT', 'MIN_TTL', 'MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN', 'MAX_IP_PKT_LEN', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES', 'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS', 'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS', 'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT', 'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES', 'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES', 'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT', 'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE', 'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label', 'Attack']


,0,1,2,3,4
L4_SRC_PORT,49235,49228,0,65317,60766
L4_DST_PORT,4444,1880,0,1900,15600
PROTOCOL,6,6,1,17,17
L7_PROTO,0.0,0.0,0.0,0.0,0.0
IN_BYTES,155392,1600,212,165,63
IN_PKTS,202,40,2,1,1
OUT_BYTES,34552,35741,0,0,0
OUT_PKTS,149,65,0,0,0
TCP_FLAGS,24,24,0,0,0
CLIENT_TCP_FLAGS,24,16,0,0,0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming df is your full NF-TON V2 dataframe
train_size = 100_000  # approx. 100k rows for Colab free tier

# Stratified sampling to preserve class proportions
df_sampled, _ = train_test_split(df, train_size=train_size, stratify=df['Label'], random_state=42)

print("Sampled dataset shape:", df_sampled.shape)


Sampled dataset shape: (100000, 43)


In [ ]:
import numpy as np

# Count NaNs per column
print("NaNs per column:\n", df_sampled.isna().sum())

# Replace infinite values with NaN
df_sampled.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN (or you can impute)
df_sampled.dropna(inplace=True)

print("Shape after removing NaNs/Infs:", df_sampled.shape)



NaNs per column:
 L4_SRC_PORT                    0
L4_DST_PORT                    0
PROTOCOL                       0
L7_PROTO                       0
IN_BYTES                       0
IN_PKTS                        0
OUT_BYTES                      0
OUT_PKTS                       0
TCP_FLAGS                      0
CLIENT_TCP_FLAGS               0
SERVER_TCP_FLAGS               0
FLOW_DURATION_MILLISECONDS     0
DURATION_IN                    0
DURATION_OUT                   0
MIN_TTL                        0
MAX_TTL                        0
LONGEST_FLOW_PKT               0
SHORTEST_FLOW_PKT              0
MIN_IP_PKT_LEN                 0
MAX_IP_PKT_LEN                 0
SRC_TO_DST_SECOND_BYTES        0
DST_TO_SRC_SECOND_BYTES        0
RETRANSMITTED_IN_BYTES         0
RETRANSMITTED_IN_PKTS          0
RETRANSMITTED_OUT_BYTES        0
RETRANSMITTED_OUT_PKTS         0
SRC_TO_DST_AVG_THROUGHPUT      0
DST_TO_SRC_AVG_THROUGHPUT      0
NUM_PKTS_UP_TO_128_BYTES       0
NUM_PKTS_128_TO_256_BYTES

In [ ]:
import numpy as np

X_raw = df_sampled.drop(columns=["Label", "Attack"]).values
y = df_sampled["Label"].values

# Clip values to avoid overflow (optional: adjust percentiles)
percentile = 99.9
upper_bounds = np.percentile(X_raw, percentile, axis=0)
lower_bounds = np.percentile(X_raw, 100 - percentile, axis=0)

X_clipped = np.clip(X_raw, lower_bounds, upper_bounds)



In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clipped)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=12)
X_pca = pca.fit_transform(X_scaled)

print("PCA output shape:", X_pca.shape)


PCA output shape: (100000, 12)


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
# Cell 5
# Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

# Function to move batch to GPU
def batch_to_device(batch, device):
    X, y = batch
    return X.to(device), y.to(device)

# DataLoaders
train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                        torch.tensor(y_train, dtype=torch.long)),
                          batch_size=16, shuffle=True)

test_loader = DataLoader(TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                       torch.tensor(y_test, dtype=torch.long)),
                         batch_size=32, shuffle=False)


In [ ]:
# Cell 6
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Weighted loss
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)



In [ ]:
# Cell 7
n_qubits = 12
num_classes = len(classes)

def create_hybrid_qnn(n_layers_var, fc1_hidden):
    dev = qml.device("default.qubit", wires=n_qubits)  # QNN on CPU

    def variational(weights):
        for layer in range(n_layers_var):
            for q in range(n_qubits):
                qml.Rot(weights[layer, q, 0], weights[layer, q, 1], weights[layer, q, 2], wires=q)
            for q in range(n_qubits):
                qml.CNOT(wires=[q, (q+1)%n_qubits])

    @qml.qnode(dev, interface="torch")
    def qnode(inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation="Y")
        variational(weights)
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

    weight_shapes = {"weights": (n_layers_var, n_qubits, 3)}
    qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

    class HybridQNN(nn.Module):
        def __init__(self):
            super().__init__()
            self.qnn = qlayer
            self.fc1 = nn.Linear(n_qubits, fc1_hidden).to(device)
            self.fc2 = nn.Linear(fc1_hidden, num_classes).to(device)

        def forward(self, x):
            x = self.qnn(x)             # QNN on CPU
            x = x.to(device)            # move output to GPU
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    return HybridQNN()


In [ ]:
# Cell 8
from tqdm import tqdm
import torch.optim as optim

def train_model(model, train_loader, val_loader, lr, epochs=3, verbose=True):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor) # Assuming class_weights_tensor is defined globally

    model.train()
    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    for epoch in range(epochs):
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        loader = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}") if verbose else train_loader

        for i, batch in enumerate(loader):
            Xb, yb = batch_to_device(batch, device)
            optimizer.zero_grad()
            out = model(Xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Calculate training accuracy
            preds = out.argmax(dim=1)
            train_correct += (preds == yb).sum().item()
            train_total += yb.size(0)


        avg_train_loss = running_loss / len(train_loader)
        train_acc = train_correct / train_total
        history["train_loss"].append(avg_train_loss)
        history["train_acc"].append(train_acc)

        # Evaluate on validation set
        val_loss, val_acc = evaluate_model_with_loss(model, val_loader, criterion)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)


        if verbose:
            print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    if verbose:
        print("Training finished.")
    return model, history

def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in loader:
            Xb, yb = batch_to_device(batch, device)
            out = model(Xb)
            preds = out.argmax(dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(yb.cpu().numpy())
    acc = np.mean(np.array(y_pred) == np.array(y_true))
    return acc

def evaluate_model_with_loss(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:
            Xb, yb = batch_to_device(batch, device)
            out = model(Xb)
            loss = criterion(out, yb)
            running_loss += loss.item()
            preds = out.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    avg_loss = running_loss / len(loader)
    acc = correct / total
    return avg_loss, acc

In [ ]:
# Cell 9
def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in loader:
            Xb, yb = batch_to_device(batch, device)
            out = model(Xb)
            preds = out.argmax(dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(yb.cpu().numpy())
    acc = np.mean(np.array(y_pred) == np.array(y_true))
    return acc


In [ ]:
# Cell 10
tuning_grid = {
    "n_layers_var": [1, 2],
    "fc1_hidden": [8, 16],
    "lr": [0.005, 0.01],
    "batch_size": [16, 32]
}

best_acc = 0
best_config = None

for n_layers_var in tuning_grid["n_layers_var"]:
    for fc1_hidden in tuning_grid["fc1_hidden"]:
        for lr in tuning_grid["lr"]:
            for batch_size in tuning_grid["batch_size"]:
                train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                                        torch.tensor(y_train, dtype=torch.long)),
                                          batch_size=batch_size, shuffle=True)

                model = create_hybrid_qnn(n_layers_var, fc1_hidden)
                model_trained = train_model(model, train_loader, lr, epochs=3, verbose=True)

                acc = evaluate_model(model_trained, test_loader)
                print(f"Layers:{n_layers_var} FC:{fc1_hidden} LR:{lr} BS:{batch_size} -> Test Acc: {acc:.4f}")

                if acc > best_acc:
                    best_acc = acc
                    best_config = {"n_layers_var": n_layers_var,
                                   "fc1_hidden": fc1_hidden,
                                   "lr": lr,
                                   "batch_size": batch_size}

print("Best hyperparameters:", best_config, "with test accuracy:", best_acc)


Epoch 1/3: 100%|██████████| 5000/5000 [06:09<00:00, 13.52it/s]


Epoch [1/3] - Average Loss: 0.5063


Epoch 2/3: 100%|██████████| 5000/5000 [06:11<00:00, 13.45it/s]


Epoch [2/3] - Average Loss: 0.4755


Epoch 3/3: 100%|██████████| 5000/5000 [06:07<00:00, 13.61it/s]


...

Epoch 1/3: 100%|██████████| 2500/2500 [06:14<00:00,  6.67it/s]


Epoch [1/3] - Average Loss: 0.4059


Epoch 2/3: 100%|██████████| 2500/2500 [06:16<00:00,  6.63it/s]


Epoch [2/3] - Average Loss: 0.3677


Epoch 3/3:  59%|█████▉    | 1476/2500 [03:36<02:26,  6.99it/s]

In [ ]:
# Cell 10
import torch
import torch.optim as optim

tuning_grid = {
    "n_layers_var": [2],
    "fc1_hidden": [8, 16],
    "lr": [0.005, 0.01],
    "batch_size": [16, 32]
}

best_acc = 0
best_config = None

for n_layers_var in tuning_grid["n_layers_var"]:
    for fc1_hidden in tuning_grid["fc1_hidden"]:
        for lr in tuning_grid["lr"]:
            for batch_size in tuning_grid["batch_size"]:
                train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                                        torch.tensor(y_train, dtype=torch.long)),
                                          batch_size=batch_size, shuffle=True)

                model = create_hybrid_qnn(n_layers_var, fc1_hidden)
                model_trained = train_model(model, train_loader, lr, epochs=3, verbose=True)

                acc = evaluate_model(model_trained, test_loader)
                print(f"Layers:{n_layers_var} FC:{fc1_hidden} LR:{lr} BS:{batch_size} -> Test Acc: {acc:.4f}")

                if acc > best_acc:
                    best_acc = acc
                    best_config = {"n_layers_var": n_layers_var,
                                   "fc1_hidden": fc1_hidden,
                                   "lr": lr,
                                   "batch_size": batch_size}

print("Best hyperparameters:", best_config, "with test accuracy:", best_acc)


Epoch 1/3: 100%|██████████| 5000/5000 [10:13<00:00,  8.15it/s]


Epoch [1/3] - Average Loss: 0.4255


Epoch 2/3: 100%|██████████| 5000/5000 [10:00<00:00,  8.33it/s]


Epoch [2/3] - Average Loss: 0.3746


Epoch 3/3: 100%|██████████| 5000/5000 [10:06<00:00,  8.24it/s]


...

Epoch [1/3] - Average Loss: 0.4258


Epoch 2/3: 100%|██████████| 2500/2500 [06:12<00:00,  6.71it/s]


Epoch [2/3] - Average Loss: 0.3744


Epoch 3/3: 100%|██████████| 2500/2500 [06:07<00:00,  6.80it/s]


Epoch [3/3] - Average Loss: 0.3648
Training finished.
Layers:2 FC:16 LR:0.01 BS:32 -> Test Acc: 0.8390
Best hyperparameters: {'n_layers_var': 2, 'fc1_hidden': 16, 'lr': 0.01, 'batch_size': 16} with test accuracy: 0.88205


In [ ]:
# Hyperparameters
n_clients = 5
rounds = 3
best_hyperparams = {'n_layers_var': 2, 'fc1_hidden': 16, 'lr': 0.01, 'batch_size': 16}

# Split dataset into clients
client_data = []
indices = np.array_split(np.arange(len(X_train)), n_clients)
for idx in indices:
    client_X = X_train[idx]
    client_y = y_train[idx]
    loader = DataLoader(TensorDataset(torch.tensor(client_X, dtype=torch.float32),
                                      torch.tensor(client_y, dtype=torch.long)),
                        batch_size=best_hyperparams['batch_size'], shuffle=True)
    client_data.append(loader)


In [ ]:
def apply_dp_gradients(model, max_norm=1.0, noise_multiplier=0.1):
    for param in model.parameters():
        if param.grad is not None:
            # Clip gradient
            torch.nn.utils.clip_grad_norm_([param], max_norm)
            # Add Gaussian noise
            param.grad += torch.randn_like(param.grad) * noise_multiplier


In [ ]:
def federated_train(model_fn, client_loaders, rounds, device, lr=0.01, dp=True):
    global_model = model_fn().to(device)

    for r in range(rounds):
        print(f"\n--- Round {r+1}/{rounds} ---")
        local_weights = []

        for i, loader in enumerate(client_loaders):
            local_model = model_fn().to(device)
            local_model.load_state_dict(global_model.state_dict())  # Start from global model
            optimizer = torch.optim.Adam(local_model.parameters(), lr=lr)
            local_model.train()

            for Xb, yb in loader:
                Xb, yb = batch_to_device((Xb, yb), device)
                optimizer.zero_grad()
                out = local_model(Xb)
                loss = criterion(out, yb)
                loss.backward()

                if dp:
                    apply_dp_gradients(local_model)

                optimizer.step()

            local_weights.append({k: v.cpu() for k, v in local_model.state_dict().items()})
            print(f"Client {i+1} done")

        # Federated averaging
        global_dict = global_model.state_dict()
        for key in global_dict.keys():
            global_dict[key] = torch.stack([w[key] for w in local_weights], 0).mean(0)
        global_model.load_state_dict(global_dict)

        # Optional: evaluate on full test set after each round
        acc = evaluate_model(global_model, test_loader)
        print(f"Round {r+1} Test Accuracy: {acc:.4f}")

    return global_model


In [ ]:
# Model factory
def model_factory():
    return create_hybrid_qnn(best_hyperparams['n_layers_var'], best_hyperparams['fc1_hidden'])

In [ ]:
global_model = federated_train(model_factory, client_data, rounds=rounds, device=device, lr=best_hyperparams['lr'], dp=True)



--- Round 1/3 ---
Client 1 done
Client 2 done
Client 3 done
Client 4 done
Client 5 done
Round 1 Test Accuracy: 0.7781

--- Round 2/3 ---
Client 1 done
Client 2 done
Client 3 done
Client 4 done
Client 5 done
Round 2 Test Accuracy: 0.8021

--- Round 3/3 ---
Client 1 done
Client 2 done
Client 3 done
Client 4 done
Client 5 done
Round 3 Test Accuracy: 0.8182


In [ ]:
global_model = federated_train(model_factory, client_data, rounds=5, device=device, lr=best_hyperparams['lr'], dp=True)


--- Round 1/5 ---
Client 1 done
Client 2 done
Client 3 done
Client 4 done
Client 5 done
Round 1 Test Accuracy: 0.7227

--- Round 2/5 ---
Client 1 done
Client 2 done
Client 3 done
Client 4 done
Client 5 done
Round 2 Test Accuracy: 0.7427

--- Round 3/5 ---
Client 1 done
Client 2 done
Client 3 done
Client 4 done
Client 5 done
Round 3 Test Accuracy: 0.8119

--- Round 4/5 ---
Client 1 done
Client 2 done
Client 3 done
Client 4 done
Client 5 done
Round 4 Test Accuracy: 0.8275

--- Round 5/5 ---
Client 1 done
Client 2 done
Client 3 done
Client 4 done
Client 5 done
Round 5 Test Accuracy: 0.8337


In [ ]:
global_model = federated_train(model_factory, client_data, rounds=10, device=device, lr=best_hyperparams['lr'], dp=True)

NameError: name 'federated_train' is not defined

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
def client_classification_report(model, client_loaders, device, classes, description=""):
    print(f"\n=== Classification Report {description} ===")

    for i, loader in enumerate(client_loaders):
        y_true, y_pred = [], []
        model.eval()
        with torch.no_grad():
            for Xb, yb in loader:
                Xb, yb = batch_to_device((Xb, yb), device)
                out = model(Xb)
                preds = out.argmax(dim=1).cpu().numpy()
                y_pred.extend(preds)
                y_true.extend(yb.cpu().numpy())
        print(f"\n--- Client {i+1} ---")
        print(classification_report(y_true, y_pred, labels=classes))


In [ ]:
# Create fresh untrained models for "before" evaluation
before_models = [model_factory().to(device) for _ in range(n_clients)]

# Evaluate each client separately using initial weights (same as global model before training)
for i in range(n_clients):
    before_models[i].load_state_dict(model_factory().state_dict())  # Untrained global model


In [ ]:
client_classification_report(model_factory(), client_data, device, classes, description="Before FedQNN Training")



=== Classification Report Before FedQNN Training ===

--- Client 1 ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4373
           1       0.73      1.00      0.84     11627

    accuracy                           0.73     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.53      0.73      0.61     16000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Client 2 ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4356
           1       0.73      1.00      0.84     11644

    accuracy                           0.73     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.53      0.73      0.61     16000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Client 3 ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4433
           1       0.72      1.00      0.84     11567

    accuracy                           0.72     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.52      0.72      0.61     16000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Client 4 ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4397
           1       0.73      1.00      0.84     11603

    accuracy                           0.73     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.53      0.73      0.61     16000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Client 5 ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4374
           1       0.73      1.00      0.84     11626

    accuracy                           0.73     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.53      0.73      0.61     16000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
client_classification_report(global_model, client_data, device, classes, description="After FedQNN Training")



=== Classification Report After FedQNN Training ===

--- Client 1 ---
              precision    recall  f1-score   support

           0       0.64      0.76      0.69      4373
           1       0.90      0.84      0.87     11627

    accuracy                           0.82     16000
   macro avg       0.77      0.80      0.78     16000
weighted avg       0.83      0.82      0.82     16000


--- Client 2 ---
              precision    recall  f1-score   support

           0       0.65      0.77      0.70      4356
           1       0.91      0.84      0.87     11644

    accuracy                           0.82     16000
   macro avg       0.78      0.81      0.79     16000
weighted avg       0.84      0.82      0.83     16000


--- Client 3 ---
              precision    recall  f1-score   support

           0       0.64      0.77      0.70      4433
           1       0.90      0.84      0.87     11567

    accuracy                           0.82     16000
   macro avg       0.

In [ ]:
# Create a fresh global model (same as used before training)
global_model_before = model_factory().to(device)

# Evaluate on test set
y_true, y_pred = [], []

global_model_before.eval()
with torch.no_grad():
    for Xb, yb in test_loader:
        Xb, yb = batch_to_device((Xb, yb), device)
        out = global_model_before(Xb)
        preds = out.argmax(dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(yb.cpu().numpy())

print("=== Global Model Before FedQNN Training ===")
print(classification_report(y_true, y_pred, labels=classes))


=== Global Model Before FedQNN Training ===
              precision    recall  f1-score   support

           0       0.27      1.00      0.43      5483
           1       0.00      0.00      0.00     14517

    accuracy                           0.27     20000
   macro avg       0.14      0.50      0.22     20000
weighted avg       0.08      0.27      0.12     20000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
y_true, y_pred = [], []

global_model.eval()
with torch.no_grad():
    for Xb, yb in test_loader:
        Xb, yb = batch_to_device((Xb, yb), device)
        out = global_model(Xb)
        preds = out.argmax(dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(yb.cpu().numpy())

print("=== Global Model After FedQNN Training ===")
print(classification_report(y_true, y_pred, labels=classes))


=== Global Model After FedQNN Training ===
              precision    recall  f1-score   support

           0       0.64      0.76      0.70      5483
           1       0.90      0.84      0.87     14517

    accuracy                           0.82     20000
   macro avg       0.77      0.80      0.78     20000
weighted avg       0.83      0.82      0.82     20000



In [ ]:
from sklearn.metrics import classification_report

def client_seen_unseen_report(model, client_loaders, device, description=""):
    print(f"\n=== {description} ===")

    for i, loader in enumerate(client_loaders):
        y_true, y_pred = [], []
        model.eval()
        with torch.no_grad():
            for Xb, yb in loader:
                Xb, yb = batch_to_device((Xb, yb), device)
                out = model(Xb)
                preds = out.argmax(dim=1).cpu().numpy()
                y_pred.extend(preds)
                y_true.extend(yb.cpu().numpy())

        seen, unseen = get_seen_unseen(y_true, y_pred, i)

        print(f"\n--- Client {i+1} ---")
        print("Seen Attacks:")
        print(classification_report(y_true, y_pred, labels=list(seen)))
        print("Unseen Attacks:")
        if unseen:
            print(classification_report(y_true, y_pred, labels=list(unseen)))
        else:
            print("No unseen attacks for this client")


In [ ]:
client_seen_unseen_report(model_factory(), client_data, device, description="Before FedQNN Training")



=== Before FedQNN Training ===

--- Client 1 ---
Seen Attacks:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4373
           1       0.73      1.00      0.84     11627

    accuracy                           0.73     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.53      0.73      0.61     16000

Unseen Attacks:
No unseen attacks for this client


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Client 2 ---
Seen Attacks:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4356
           1       0.73      1.00      0.84     11644

    accuracy                           0.73     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.53      0.73      0.61     16000

Unseen Attacks:
No unseen attacks for this client


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Client 3 ---
Seen Attacks:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4433
           1       0.72      1.00      0.84     11567

    accuracy                           0.72     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.52      0.72      0.61     16000

Unseen Attacks:
No unseen attacks for this client


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Client 4 ---
Seen Attacks:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4397
           1       0.73      1.00      0.84     11603

    accuracy                           0.73     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.53      0.73      0.61     16000

Unseen Attacks:
No unseen attacks for this client


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Client 5 ---
Seen Attacks:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4374
           1       0.73      1.00      0.84     11626

    accuracy                           0.73     16000
   macro avg       0.36      0.50      0.42     16000
weighted avg       0.53      0.73      0.61     16000

Unseen Attacks:
No unseen attacks for this client


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
client_seen_unseen_report(global_model, client_data, device, description="After FedQNN Training")



=== After FedQNN Training ===

--- Client 1 ---
Seen Attacks:
              precision    recall  f1-score   support

           0       0.64      0.76      0.69      4373
           1       0.90      0.84      0.87     11627

    accuracy                           0.82     16000
   macro avg       0.77      0.80      0.78     16000
weighted avg       0.83      0.82      0.82     16000

Unseen Attacks:
No unseen attacks for this client

--- Client 2 ---
Seen Attacks:
              precision    recall  f1-score   support

           0       0.65      0.77      0.70      4356
           1       0.91      0.84      0.87     11644

    accuracy                           0.82     16000
   macro avg       0.78      0.81      0.79     16000
weighted avg       0.84      0.82      0.83     16000

Unseen Attacks:
No unseen attacks for this client

--- Client 3 ---
Seen Attacks:
              precision    recall  f1-score   support

           0       0.64      0.77      0.70      4433
          

In [ ]:
# Print first 5 rows
print(X_pca[:5, :])


[[ 1.94529362e+00 -1.38085651e+00 -2.50262658e-01  6.69346535e-01
   4.73139760e-01 -1.12968948e+00  1.21173417e+00  8.57921520e-01
   1.85925696e-01  8.88823750e-02 -2.22233683e-02  5.73551146e-01]
 [-2.64042908e+00  2.50419504e-01 -1.03785814e+00 -3.96521344e-01
  -1.72508375e+00  4.29173451e-01 -1.95576691e-01 -6.35014618e-01
  -6.30823395e-02  2.25009764e-02  6.91065682e-02  2.05373530e-01]
 [-1.67492670e+00 -2.72144015e-01 -1.13939821e+00 -1.08120825e-01
  -1.38523000e+00 -3.77471482e-01 -2.72492876e-01 -3.63326288e-01
  -1.11945589e-01  5.89882466e-01 -1.31583049e-01  1.00166369e+00]
 [ 2.92052184e-01 -1.22111828e+00 -8.25567722e-01  3.05787613e-01
   5.07863416e-01 -8.49677728e-01  2.21302700e-01  1.06832920e+00
   3.23845034e-01 -2.73673471e-01  1.45053161e-01  2.91073270e-01]
 [-2.89814368e+00  1.12446145e+00  2.40144260e+00  1.15359604e-02
   1.50625130e+00 -6.95319223e-02 -1.95755140e-02 -4.15349461e-02
  -3.79883783e-02  1.81759436e-03 -1.30689819e-01  2.01603096e-02]]


In [ ]:
def apply_dp_gradients(model, max_norm=1.0, noise_multiplier=0.1, device='cpu'):
    """
    Apply Differential Privacy to gradients:
    - Clips gradients per parameter
    - Adds Gaussian noise
    This is a simple DP mechanism suitable for federated learning.
    """
    for param in model.parameters():
        if param.grad is not None:
            # Clip gradient per parameter
            param_norm = param.grad.data.norm(2)
            clip_coef = max_norm / (param_norm + 1e-6)
            if clip_coef < 1:
                param.grad.data.mul_(clip_coef)

            # Add Gaussian noise
            noise = torch.randn_like(param.grad).to(device) * noise_multiplier
            param.grad.data.add_(noise)


In [ ]:
def federated_train(model_fn, client_loaders, rounds, device, lr=0.01, dp=True):
    global_model = model_fn().to(device)

    for r in range(rounds):
        print(f"\n--- Round {r+1}/{rounds} ---")
        local_weights = []

        for i, loader in enumerate(client_loaders):
            local_model = model_fn().to(device)
            local_model.load_state_dict(global_model.state_dict())  # Start from global model
            optimizer = torch.optim.Adam(local_model.parameters(), lr=lr)
            local_model.train()

            for Xb, yb in loader:
                Xb, yb = batch_to_device((Xb, yb), device)
                optimizer.zero_grad()
                out = local_model(Xb)
                loss = criterion(out, yb)
                loss.backward()

                if dp:
                    apply_dp_gradients(local_model, max_norm=1.0, noise_multiplier=0.1, device=device)

                optimizer.step()

            local_weights.append({k: v.cpu() for k, v in local_model.state_dict().items()})
            print(f"Client {i+1} done")

        # Federated averaging
        global_dict = global_model.state_dict()
        for key in global_dict.keys():
            global_dict[key] = torch.stack([w[key] for w in local_weights], 0).mean(0)
        global_model.load_state_dict(global_dict)

        # Optional: evaluate on full test set after each round
        acc = evaluate_model(global_model, test_loader)
        print(f"Round {r+1} Test Accuracy: {acc:.4f}")

    return global_model

In [ ]:
global_model_dp = federated_train(model_factory, client_data, rounds=rounds, device=device, lr=best_hyperparams['lr'], dp=True)

NameError: name 'model_factory' is not defined

In [ ]:
def federated_train(model_fn, client_loaders, rounds, device, lr=0.01, dp=True, initial_model=None, return_history=True):
    """
    Federated training with optional DP, continued training, and round-wise history logging.

    Args:
        model_fn: function that returns a new model instance
        client_loaders: list of DataLoaders for each client
        rounds: number of federated rounds
        device: torch device
        lr: learning rate
        dp: whether to apply differential privacy
        initial_model: optional pre-trained global model to continue training
        return_history: whether to return round-wise accuracy and loss
    Returns:
        global_model: trained global model
        history (optional): dict containing round-wise accuracy and optionally loss
    """
    # Initialize model
    if initial_model is not None:
        global_model = initial_model.to(device)
    else:
        global_model = model_fn().to(device)

    history = {
        "round": [],
        "test_acc": [],
        "client_train_acc": []
    }

    for r in range(rounds):
        print(f"\n--- Round {r+1}/{rounds} ---")
        local_weights = []
        round_client_acc = []

        for i, loader in enumerate(client_loaders):
            local_model = model_fn().to(device)
            local_model.load_state_dict(global_model.state_dict())  # Start from current global model
            optimizer = torch.optim.Adam(local_model.parameters(), lr=lr)
            local_model.train()

            for Xb, yb in loader:
                Xb, yb = batch_to_device((Xb, yb), device)
                optimizer.zero_grad()
                out = local_model(Xb)
                loss = criterion(out, yb)
                loss.backward()

                if dp:
                    apply_dp_gradients(local_model, max_norm=1.0, noise_multiplier=0.1, device=device)

                optimizer.step()

            # Save local model weights (CPU to save memory)
            local_weights.append({k: v.cpu() for k, v in local_model.state_dict().items()})

            # Optional: compute training accuracy for this client
            client_acc = evaluate_model(local_model, loader)
            round_client_acc.append(client_acc)
            print(f"Client {i+1} done - Train Acc: {client_acc:.4f}")

        # Federated averaging
        global_dict = global_model.state_dict()
        for key in global_dict.keys():
            global_dict[key] = torch.stack([w[key] for w in local_weights], 0).mean(0)
        global_model.load_state_dict(global_dict)

        # Evaluate on global test set
        test_acc = evaluate_model(global_model, test_loader)
        print(f"Round {r+1} Test Accuracy: {test_acc:.4f}")

        # Save history
        history["round"].append(r+1)
        history["test_acc"].append(test_acc)
        history["client_train_acc"].append(round_client_acc)

    if return_history:
        return global_model, history
    else:
        return global_model


In [ ]:
global_model_dp, history1 = federated_train(
    model_fn=model_factory,
    client_loaders=client_data,
    rounds=3,
    device=device,
    lr=best_hyperparams['lr'],
    dp=True
)


--- Round 1/3 ---
Client 1 done - Train Acc: 0.4774
Client 2 done - Train Acc: 0.4918
Client 3 done - Train Acc: 0.4884
Client 4 done - Train Acc: 0.4575
Client 5 done - Train Acc: 0.5332
Round 1 Test Accuracy: 0.4009

--- Round 2/3 ---
Client 1 done - Train Acc: 0.5385
Client 2 done - Train Acc: 0.5263
Client 3 done - Train Acc: 0.5072
Client 4 done - Train Acc: 0.5548
Client 5 done - Train Acc: 0.5136
Round 2 Test Accuracy: 0.5087

--- Round 3/3 ---
Client 1 done - Train Acc: 0.5652
Client 2 done - Train Acc: 0.5935
Client 3 done - Train Acc: 0.5960
Client 4 done - Train Acc: 0.5887
Client 5 done - Train Acc: 0.5706
Round 3 Test Accuracy: 0.5701
